In [1]:
import os
import sys
import pandas as pd
import numpy as np
import regex as re
import mygene
from matplotlib import pyplot as plt

In [2]:
mg = mygene.MyGeneInfo()

In [3]:
mg.getgene("ENSG00000198938", 'name,symbol,refseq.rna,type_of_gene,exac.bp')

{u'_id': u'4514',
 u'_score': 11.708687,
 u'name': u'cytochrome c oxidase III',
 u'symbol': u'COX3',
 u'type_of_gene': u'protein-coding'}

In [4]:
working_dir = "/Users/filippo/Developer/tesi"
os.chdir(working_dir)
dirs = os.listdir("data")

In [5]:
len(dirs)

8935

In [10]:
df = pd.read_csv(("%s/genes.txt"%working_dir), header=None)
df.columns=["gene"]
print(len(df['gene']))
df.info()

60483
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60483 entries, 0 to 60482
Data columns (total 1 columns):
gene    60483 non-null object
dtypes: object(1)
memory usage: 472.6+ KB


In [11]:
uniq = [g[:15] for g in df['gene'].values]

In [12]:
len(np.unique(uniq))

60483

In [13]:
df = pd.read_csv(("%s/mainTable.csv"%working_dir))
genes = np.array([gene[:15] for gene in df.loc[:,'gene'].values])
df['gene'] = genes
df = df.to_sparse(fill_value=0.)

In [14]:
ngenes = len(df['gene'])
nfiles = len(df.loc[0,:])-1
print("genes:%d\trealizations:%d"%(ngenes,nfiles))

genes:60483	realizations:5000


## Means sigmas
thresholds are applied for FPKM between 0.1 and 1e5

In [ ]:
%timeit
means = []
for i,g in enumerate(df.loc[:,df.keys()[0]].values):
    means.append(np.average(df.loc[i,df.keys()[1:]].values))
    if(i%100==0):
        print("elapsed %d"%i)

In [ ]:
%timeit
variances = []
for i,g in enumerate(df.loc[:,df.keys()[0]].values):
    variances.append(np.var(df.loc[i,df.keys()[1:]].values))
    if(i%100==0):
        print("elapsed %d"%i)

In [ ]:
df_mv = pd.DataFrame(data=np.array(means), columns=['mean'], index=df.loc[:,df.keys()[0]].values)
df_mv.insert(1, 'variance', np.array(variances))
df_mv.head()
df_mv.to_csv("meanVariances.csv")

In [45]:
df_mv = pd.read_csv("meanVariances.csv", index_col = [0])
df_mv.head()

,mean,variance,type_of_gene
gene,,,
ENSG00000242268.2,0.614475,0.894804,NaN
ENSG00000270112.3,0.345251,0.160901,NaN
ENSG00000167578.15,3.861120,6.344060,NaN
ENSG00000273842.1,0.436278,0.193080,NaN
ENSG00000078237.5,4.459130,17.956900,NaN


In [ ]:
gdf = pd.read_csv("genes.csv")
gdf.head()

In [ ]:
df_mv = df_mv.drop(labels='type_of_gene')
df_mv.insert(1,'type_of_gene', '')
for g in df_mv.index.values:
    df_mv.at[g, 'type_of_gene'] = gdf.loc[g[:15]]['type_of_gene']